In [ ]:
import os
from glob import glob

import numpy as np
from mpl_toolkits import mplot3d
from matplotlib import pyplot
from tqdm import tqdm

from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist

import trimesh
#requires shapely, rtree, networkx, pyglet

import mbb

In [ ]:
"""
This notebook is for a new method to simulate the manual measurements of hailstones on 3D models.
This includes the measurement of Dmax, Dint and Dmin following the standard procedure.
The implementation is heavily optimised to use the trimesh library and convexhulls.

Summary of method:

(1) Shift model centre of mass to global coordinates 0,0,0
(2) calculate 3D convex hull of model
(3) calculate which pair of vertices in 3D convex hull have the greatest separation (Dmax)
(4) Calculate normal plane to Dmax (Dint-Dmin plane), and the mid point
(5) Slice the convex hull at this mid point using the Dint-Dmin plane
(6) Fit a 2D minimum bounded box to the convex hull slice to find Dint and Dmin.

"""

'\nThis repo is for a new method to simulate the manual measurements of hailstones using 3D models.\nThis includes the measurement of Dmax, Dint and Dmin following the standard procedure.\nThe implementation is heavily optimised to use the trimesh library and convexhulls.\n\nSummary of method:\n\n(1) Shift model centre of mass to 0,0,0\n(2) calculate convex hull of model\n(3) calculate which pair of vertices have the greatest separation\n(4) Calculate normal vector of plane normal to the Dmax (Dint-Dmin plane), and the mid point\n(5) Slice the convex hull at this mid point using the Dint-Dmin plane\n(6) Fit a minimum bounded box to the slice to find Dint and Dmin.\n\nTODO: Proper output for analysis of errors\n'

In [2]:
def slice_midpoint_and_measure(tmp_mesh, midpoint, nvec, color):

    # slice mesh at midpoint along normal vector,
    # fit with bounding box and measure minimum (Dmin) and maximum (Dint) 

    #create slice
    slice = tmp_mesh.section(plane_origin=midpoint, 
                        plane_normal=nvec)
    #convert to 2D projection
    slice_2D, to_3D = slice.to_planar(normal=nvec)
    #set colour of slice
    slice.colors = [color] * len(slice.entities)
    #generate bounding box on 2D projection
    bounding_box_stats = mbb.MinimumBoundingBox(slice_2D.vertices)
    #use bounding box to calculate Dint and Dmin
    Dint = bounding_box_stats.length_parallel
    Dmin = bounding_box_stats.length_orthogonal
    #extract corner points of bounding box and order
    corner_points = np.array(list(bounding_box_stats.corner_points))
    pts_order = trimesh.points.tsp(corner_points, start=0)[0]
    corner_points = corner_points[pts_order]
    #create a new 2Dpath object for corner points, using transform from slice
    slice_bb = trimesh.path.Path2D(entities=[trimesh.path.entities.Line([0,1,2,3,0])], vertices=corner_points, colors=[color]).to_3D(transform=to_3D)

    return slice, slice_bb, Dint, Dmin


def measure_shape(stl_ffn):
    #load mesh
    mymesh = trimesh.load_mesh(stl_ffn)
    # volumetric center of mass which we can set as the origin for our mesh
    mymesh.vertices -= mymesh.center_mass
    if not mymesh.is_watertight:
        print('WARNING, MESH is not watertight!, stats may be misleading')
    #stats
    com = mymesh.center_mass
    volume = mymesh.volume
    mymesh_hull = mymesh.convex_hull
    mymesh_hull.visual.face_colors = [0,0,0,50]
    mymesh.visual.vertex_colors = [255,255,255,255]
    
    print("transparency", mymesh.visual.transparency)
    # Naive way of finding the best pair in O(H^2) time if H is number of points on hull
    Dmax_hdist = cdist(mymesh_hull.vertices, mymesh_hull.vertices, metric='euclidean')
    Dmax = Dmax_hdist.max()
    # Get the farthest apart points
    Dmax_bestpair = np.unravel_index(Dmax_hdist.argmax(), Dmax_hdist.shape)
    Dmax_points = [mymesh_hull.vertices[Dmax_bestpair[0]], mymesh_hull.vertices[Dmax_bestpair[1]]]

    #calculate mid point and normal vector
    Dmax_midpoint = (Dmax_points[0] + Dmax_points[1])/2
    #print(Dmax_midpoint)
    #calculate vector between Dmax points
    Dint_Dmin_plane_nvec = Dmax_points[0] - Dmax_points[1]
    

    #1: slice along Dint/Dmin plane
    myslice, slice_bb, Dint, Dmin = slice_midpoint_and_measure(mymesh, Dmax_midpoint, Dint_Dmin_plane_nvec, [255, 0, 0, 255]) #red
    #2: slice convex hull along Dint/Dmin plane
    myslice_hull, slice_bb_hull, Dint_hull, Dmin_hull = slice_midpoint_and_measure(mymesh_hull, Dmax_midpoint, Dint_Dmin_plane_nvec, [0,0,255,255]) #blue
    #3: slice along Dmax/Dmin plane
    Dmax_Dmin_plane_nvec = slice_bb.vertices[0,:] - slice_bb.vertices[1,:]
    myslice_dmax, _, _, _ = slice_midpoint_and_measure(mymesh, Dmax_midpoint, Dmax_Dmin_plane_nvec, [0, 255, 0, 255]) #green

    # stack rays into line segments for visualization as Path3D
    dmax_ray = trimesh.load_path(np.vstack([Dmax_points[0],
                                             Dmax_points[1]])*1.05)



    return Dmax, Dint, Dmin, mymesh, myslice, slice_bb, mymesh_hull, myslice_hull, slice_bb_hull, myslice_dmax, dmax_ray

In [3]:
stl_ffn_list = sorted(glob('/g/data/kl02/jss548/hail_measurement_error_models/*.stl'))
print(stl_ffn_list)
for stl_ffn in stl_ffn_list:
    Dmax, Dint, Dmin, mymesh, myslice, slice_bb, mymesh_hull, myslice_hull, slice_bb_hull, myslice_dmax, dmax_ray = measure_shape(stl_ffn)
    print(stl_ffn, int(Dmax), int(Dint), int(Dmin))
    #break

['/g/data/kl02/jss548/hail_measurement_error_models/8a1-1 studio famous mini stone.stl', '/g/data/kl02/jss548/hail_measurement_error_models/bergen_model_03.stl', '/g/data/kl02/jss548/hail_measurement_error_models/gatton_20231223_2-8.stl', '/g/data/kl02/jss548/hail_measurement_error_models/melb-20200119-20-edit.stl', '/g/data/kl02/jss548/hail_measurement_error_models/melb_20200119-01-edit.stl']
transparency False


/scratch/kl02/jss548/tmp/ipykernel_1525144/121607600.py:10: DeprecationWarning: DEPRECATED: replace `path.to_planar`->`path.to_2D), removal 1/1/2026
  slice_2D, to_3D = slice.to_planar(normal=nvec)


/g/data/kl02/jss548/hail_measurement_error_models/8a1-1 studio famous mini stone.stl 37 17 16
transparency False
/g/data/kl02/jss548/hail_measurement_error_models/bergen_model_03.stl 69 62 64
transparency False
/g/data/kl02/jss548/hail_measurement_error_models/gatton_20231223_2-8.stl 71 42 58
transparency False
/g/data/kl02/jss548/hail_measurement_error_models/melb-20200119-20-edit.stl 39 35 14
WARNING, MESH is not watertight!, stats may be misleading
transparency False
/g/data/kl02/jss548/hail_measurement_error_models/melb_20200119-01-edit.stl 51 46 43


In [99]:
#set scence for visualisation and export:

scene_list = [mymesh]
scene = trimesh.Scene(scene_list)
print(scene.lights)
scene.show()